# PyTorch Dataset

In [64]:
import os
import sys
from importlib import reload
sys.path.append('../src')

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import tqdm

In [65]:
import transformers

In [2]:
import torch.utils.data as D

In [120]:
import dataset
import transforms

In [183]:
reload(dataset)
valid = dataset.Dataset('../input/validation.npz')

In [10]:
tokenizer = transformers.XLMRobertaTokenizer.from_pretrained('xlm-roberta-large')

## Sampling

We will create weighted random sampler that will produce random samples with replacement for each epoch with approximately equal amount of positiva and negative samples

In [184]:
sampler = valid.weighted_sampler()

In [205]:
# rerun this cell to ensure that values will be different for each epoch
sampled = [i for i in sampler]
np.unique(valid.y[sampled], return_counts=True), len(sampled)

((array([0, 1], dtype=uint8), array([1228, 1232])), 2460)

## Transforms

In [146]:
reload(transforms)

<module 'transforms' from '../src/transforms.py'>

In [131]:
shuffle_sent = transforms.ShuffleSentences(0.5)

In [132]:
%timeit _ = shuffle_sent(valid[12][0])

17.9 µs ± 3.14 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [133]:
shuffle_word = transforms.SwapWords(0.1)

In [134]:
%timeit _ = shuffle_word(valid[12][0])

1.01 ms ± 43.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [135]:
drop_word = transforms.DropWords(0.1)

In [136]:
%timeit _ = drop_word(valid[12][0])

1.19 ms ± 171 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [147]:
all_transforms = transforms.Compose([
    transforms.ShuffleSentences(0.25),
    transforms.SwapWords(0.1),
    transforms.DropWords(0.1),
    transforms.ToTensor(),
])

In [148]:
all_transforms.transforms

In [149]:
%timeit _ = all_transforms(valid[12][0])

2.33 ms ± 421 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [79]:
tokenizer.decode(shuffle(valid[12][0]))

'<s>Mr. No sabemos quien es el autor del pie de imagen por ende no podemos acusarlo de haber inventado o no inventado la afirmacion, ergo no podemos decir que es investigacion original ya que no sabemos de donde obtuvo las lineas que escribió, lo que si sabemos es que la carga de la prueba recae sobre quien hace la afirmación, y en este caso es Xana la que afirma que el pie de imagen es subjetivo, pero, ¿segun quien?¿segun ella misma?, entonces en ese caso decir que el pie de iamgen es subjetivo es una afirmacion subjetiva de la propia subjetividad de Xana ya que no ha demostrado pruebas objetivas de que la susodicha sea realmente subjetiva, ergo eliminar tanto el pie de foto como la imagen misma nomás porque así lo dice la subjetividad de Xana, eso sería precisamente una edición subjetiva carente de argumentacion enciclopédica y por eso procedí a revertirla.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

In [117]:
tokenizer.decode(drop_word(valid[12][0]))

'<s> No quien el del pie de imagen ende no podemos acusarlo haber invent o no inventado afirma, ergo podemos decir escion original que sabemos de ob lass escribió lo si es la carga de la pruebae quien lación, en caso esa la que que el de imagen esje,,segun quien¿segu ella misma? entonces ese decir el pie degen es subtivo una afirma subtiva de la propia subjetiv de Xana ya que no demostrado objetiva de que ladi sea realmenteje, eliminar el de como imagen nomás porque así lo dice lajetiv dea eso sería una edición subtiva care dea enpé y eso proced a reverlaMr.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p

In [80]:
tokenizer.decode(valid[12][0])

'<s> No sabemos quien es el autor del pie de imagen por ende no podemos acusarlo de haber inventado o no inventado la afirmacion, ergo no podemos decir que es investigacion original ya que no sabemos de donde obtuvo las lineas que escribió, lo que si sabemos es que la carga de la prueba recae sobre quien hace la afirmación, y en este caso es Xana la que afirma que el pie de imagen es subjetivo, pero, ¿segun quien?¿segun ella misma?, entonces en ese caso decir que el pie de iamgen es subjetivo es una afirmacion subjetiva de la propia subjetividad de Xana ya que no ha demostrado pruebas objetivas de que la susodicha sea realmente subjetiva, ergo eliminar tanto el pie de foto como la imagen misma nomás porque así lo dice la subjetividad de Xana, eso sería precisamente una edición subjetiva carente de argumentacion enciclopédica y por eso procedí a revertirla.Mr.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>